In [14]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


Importing

In [41]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma


from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

Data Loading

In [16]:
loader = Docx2txtLoader(r'/Users/aungkaungpyaepaing/Desktop/df/Introduction_to_Data_and_Data_Science_2.docx')
pages = loader.load()

Maskdown Split

In [17]:
md_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=[('#', 'Course Title'),
                                                              ('##', 'Lecture Title')])
pages_md_split = md_splitter.split_text(pages[0].page_content)

Removing new line /n from text

In [18]:
for i in range(len(pages_md_split)):
    pages_md_split[i].page_content = ' '.join(pages_md_split[i].page_content.split())

Character Splitter

In [19]:
char_split = CharacterTextSplitter(
    separator='.',
    chunk_size=500,
    chunk_overlap=50
)

In [20]:
pages_char_split = char_split.split_documents(pages_md_split)

Embedding

In [21]:
embedding = OpenAIEmbeddings(model = 'text-embedding-ada-002')

Vector Storing using Chroma

In [23]:
vectorstore = Chroma.from_documents(documents = pages_char_split, embedding = embedding, persist_directory= './intro-ds-vectorstore')

In [25]:
vectorstore_from_directory = Chroma(persist_directory = './intro-ds-vectorstore',
                                    embedding_function=embedding)

In [27]:
len(vectorstore_from_directory.get()['documents'])

20

Retriever

In [35]:
retriever = vectorstore_from_directory.as_retriever(serch_type = 'mmr',
                                                    search_kwargs = {'k':3})

setting up Chain

In [36]:
TEMPLATE = '''
Answer the following question:
{question}

To answer the question, use only the following context:
{context}

At the end of the response, specify the name of the lecture this context is taken from in the format:
Resources : *Lecture Title*
where *Lecture Title* should be substituted with the title all resource lectures.
'''

prompt_template = PromptTemplate.from_template(TEMPLATE)

In [37]:
chat = ChatOpenAI(model_name = 'gpt-4',
                  model_kwargs={'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/Users/aungkaungpyaepaing/PycharmProjects/LangChain/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3610: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [38]:
question = "Which programming language do data scientists use?"

Building Chain

In [39]:
chain = ({'context':retriever,
         'question':RunnablePassthrough()}
         | prompt_template
         | chat
         | StrOutputParser())


In [40]:
chain.invoke(question)

'Data scientists use several programming languages. The most popular ones are R and Python, which are highly adaptable and can manipulate data across multiple data and data science software platforms. They are suitable for mathematical and statistical computations and can solve a wide variety of business and data-related problems. Apart from R and Python, data scientists also use other languages like Java, Scala, MATLAB, JavaScript, C, and C++. These languages are particularly useful when dealing with big data and machine learning, where a lot of computational power'